#4 Class-Classification
## ImageNet
## first model  >> baseline_resnet50

In [4]:
import numpy as np
import urllib.request
import cv2

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
img_rows, img_cols = 224, 224 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last

In [8]:
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
	return image

In [9]:
#%clear base_model

import tensorflow as tf

base_model = tf.keras.applications.ResNet50(input_shape=(224,224, 3),weights='imagenet', include_top = False)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
preds = tf.keras.layers.Dense(4, activation ='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=preds)

In [10]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
    
train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Monkeys_Species_dataset/training',
        classes=['blad_uakari','black_headed_night_monkey','japanese_macaque','patas_monkey'],
        target_size=(img_rows, img_cols),#The target_size is the size of your input images,every image will be resized to this size
        batch_size=32,
        class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
        '/content/drive/MyDrive/Monkeys_Species_dataset/validation',
        classes=['blad_uakari','black_headed_night_monkey','japanese_macaque','patas_monkey'],        
        target_size=(img_rows, img_cols),#The target_size is the size of your input images,every image will be resized to this size
        batch_size=32,
        class_mode='categorical')

Found 339 images belonging to 4 classes.
Found 85 images belonging to 4 classes.


In [11]:
for layer in base_model.layers:
  layer.trainable = False

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.n//train_generator.batch_size,
      epochs=5,
      validation_data=valid_generator,
      validation_steps=25)

Epoch 1/5
10/10 [==============================] - 87s 8s/step - loss: 2.0946 - accuracy: 0.4593 - val_loss: 0.3983 - val_accuracy: 0.8706
Epoch 2/5
10/10 [==============================] - 7s 687ms/step - loss: 0.1331 - accuracy: 0.9812
Epoch 3/5
10/10 [==============================] - 7s 667ms/step - loss: 0.1650 - accuracy: 0.9642
Epoch 4/5
10/10 [==============================] - 7s 642ms/step - loss: 0.2203 - accuracy: 0.9479
Epoch 5/5
10/10 [==============================] - 7s 702ms/step - loss: 0.2464 - accuracy: 0.9381


In [14]:
test_datagen = ImageDataGenerator()

In [16]:
test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Monkeys_Species_dataset/training',
        classes=['blad_uakari','black_headed_night_monkey','japanese_macaque','patas_monkey'],
        target_size=(374, 374),#The target_size is the size of your input images,every image will be resized to this size
        batch_size=32,
        class_mode='categorical')

Found 339 images belonging to 4 classes.


In [17]:
history=model.evaluate(
test_generator    
)

11/11 [==============================] - 11s 851ms/step - loss: 0.3818 - accuracy: 0.8289


In [18]:
print(history)

[0.3818231523036957, 0.8289085626602173]


In [19]:
print(test_generator)